### Query

In [1]:
import os

In [6]:
query = '?$filter=substringof(%2743SFR%27,Name) and Online eq true'
username = 'danyayay'
password = 'Ldy19970722'
output_doc = '../data/q.txt'

In [9]:
url_query = "https://scihub.copernicus.eu/dhus/odata/v1/Products" + query
cmd_query = f'wget --no-check-certificate --user={username} --password={password} --output-document={output_doc} "{url_query}"'
os.system(cmd_query)

0

### Download

In [18]:
id_ = '2c50ca48-661e-4163-ba0a-902c4811cf09' # offline

In [5]:
url = f"https://scihub.copernicus.eu/dhus/odata/v1/Products('{id_}')/$value"

In [6]:
os.system(f'wget --content-disposition --continue --user=danyayay --password=Ldy19970722 "{url}"')

0

In [19]:
url_online = f"https://scihub.copernicus.eu/dhus/odata/v1/Products('{id_}')/Online"

In [20]:
os.popen(f'wget --user=danyayay --password=Ldy19970722 "{url_online}"').read()

''

In [7]:
id_ = '0e37a78b-3b91-4c15-bef2-33c2082044a7' # online

In [8]:
url = f"https://scihub.copernicus.eu/dhus/odata/v1/Products('{id_}')/$value"

In [9]:
os.system(f'wget --content-disposition --continue --user=danyayay --password=Ldy19970722 "{url}"')

0

### Extract

In [36]:
with open('../data/output.txt') as f:
    lines = f.readlines()

In [46]:
query_data = ''
for l in lines:
    query_data += l

In [38]:
import re 

In [60]:
ids = re.findall('<m:properties><d:Id>(.*)</d:Id><d:Name>(.*)</d:Name>.*<d:Online>(.*)</d:Online>', query_data)

In [67]:
ids

[('e55ece38-f77a-4953-865b-d570c9997aa9',
  'S2B_MSIL2A_20210616T053639_N0300_R005_T43SFR_20210616T082742',
  'true'),
 ('5b4a6ce4-7356-4baf-974b-36335d49b367',
  'S2B_MSIL2A_20191211T053219_N0213_R105_T43SFR_20191211T085824',
  'true'),
 ('2c50ca48-661e-4163-ba0a-902c4811cf09',
  'S2A_MSIL1C_20200603T052651_N0209_R105_T43SFR_20200603T091952',
  'true'),
 ('923d53df-40d3-4df7-84c7-90d47fda5f5d',
  'S2A_MSIL1C_20210728T052651_N0301_R105_T43SFR_20210728T082347',
  'true'),
 ('8d30e360-13a5-48fe-94bc-7e746a420943',
  'S2B_MSIL2A_20210626T053639_N0300_R005_T43SFR_20210626T083628',
  'true'),
 ('9d8960ee-3882-4559-b055-4a86e94de2ba',
  'S2A_MSIL1C_20210621T053641_N0300_R005_T43SFR_20210621T075059',
  'true'),
 ('16241b12-cce4-4bf4-8e55-5a790134a1d0',
  'S2A_MSIL1C_20210807T052651_N0301_R105_T43SFR_20210807T073546',
  'true'),
 ('1b6a063d-c459-4696-8aad-1a7be0927959',
  'S2A_MSIL2A_20210621T053641_N0300_R005_T43SFR_20210621T084400',
  'true'),
 ('93b26afa-9d74-4cff-b647-a5b5e043b49e',
  'S2A

In [77]:
ids[-2][1]

'S2A_MSIL2A_20210529T052651_N0300_R105_T43SFR_20210529T075026'

In [75]:
ids[-1][1].startswith('S2A_MSIL1C_2021')

False

In [59]:
len(ids)

50

Download data using wget

In [1]:
import os

In [4]:
url1 = "https://scihub.copernicus.eu/dhus/odata/v1/Products('22e7af63-07ad-4076-8541-f6655388dc5e')/\$value"

In [6]:
url2 = "https://scihub.copernicus.eu/dhus/odata/v1/Products('2b17b57d-fff4-4645-b539-91f305c27c69')/$value"

In [5]:
url3 = "https://scihub.copernicus.eu/dhus/odata/v1/Products('2aa207f9-608c-4548-b992-2c47bcd5793c')/$value"

In [6]:
os.system(f'wget  --content-disposition --continue --user=danyayay --password=Ldy19970722 "{url3}"')

0

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('2aa207f9-608c-4548-b992-2c47bcd5793c')</id><title type="text">S2A_MSIL2A_20201220T053231_N0214_R105_T43SFR_20201220T075655</title><updated>2021-07-23T19:19:53.987Z</updated><category term="DHuS.Product" scheme="http://schemas.microsoft.com/ado/2007/08/dataservices/scheme"/><link href="Products(&apos;2aa207f9-608c-4548-b992-2c47bcd5793c&apos;)" rel="edit" title="Product"/>

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('2c50ca48-661e-4163-ba0a-902c4811cf09')/Online/$value

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('2c50ca48-661e-4163-ba0a-902c4811cf09')/$value

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('2c50ca48-661e-4163-ba0a-902c4811cf09')/Nodes('S2A_MSIL1C_20200603T052651_N0209_R105_T43SFR_20200603T091952.SAFE')/Nodes('manifest.safe')/$value

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('0a91217c-c602-471c-bda1-4926af39840f')/Nodes('S2A_MSIL1C_20210509T052651_N0300_R105_T43SFR_20210509T074019.SAFE')/Nodes('GRANULE')/$value

good

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('2b17b57d-fff4-4645-b539-91f305c27c69')/Nodes('S1A_IW_SLC__1SDV_20160117T103451_20160117T103518_009533_00DD94_D46A.SAFE')/Nodes('manifest.safe')/$value

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('ac8b1986-73e7-4ed2-b21a-8cfefead7e7b')/$value

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('ac8b1986-73e7-4ed2-b21a-8cfefead7e7b')/Nodes('S2A_MSIL1C_20210601T053641_N0300_R005_T43SFR_20210601T074909.SAFE')/Nodes('manifest.safe')/$value

In [ ]:
https://scihub.copernicus.eu/dhus/odata/v1/Products('8bd3bef3-6a11-4111-ba87-fd3004ed47fd')/Nodes('S2B_MSIL2A_20210523T210019_N0300_R100_T07WDQ_20210523T234058.SAFE')/Nodes('manifest.safe')/$value

Download using urllib

In [7]:
import urllib

In [9]:
# create a password manager
password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()

# Add the username and password.
# If we knew the realm, we could use it instead of None.
top_level_url = "https://scihub.copernicus.eu"
password_mgr.add_password(None, top_level_url, 'danyayay', 'Ldy19970722')

handler = urllib.request.HTTPBasicAuthHandler(password_mgr)

# create "opener" (OpenerDirector instance)
opener = urllib.request.build_opener(handler)

# use the opener to fetch a URL
a_url = top_level_url + "/dhus/odata/v1/Products('16241b12-cce4-4bf4-8e55-5a790134a1d0')/$value"
opener.open(a_url)

# Install the opener.
# Now all calls to urllib.request.urlopen use our opener.
urllib.request.install_opener(opener)

In [10]:
import requests

In [11]:
theurl= 'https://scihub.copernicus.eu/dhus/odata/v1/Products('16241b12-cce4-4bf4-8e55-5a790134a1d0')/$value'
username = 'myUsername'
password = 'myPassword'

r=requests.get(theurl, auth=requests.auth.HTTPBasicAuth(username, password))

MissingSchema: Invalid URL 'myLink_queriedResult/result.xls': No schema supplied. Perhaps you meant http://myLink_queriedResult/result.xls?